# Cell Tracking and Lineage Construction in Live-Cell Imaging Data

### Part 2 (of 2)
---

## Employing the Model to Track Cells and Benchmarking the Results
---

Implementation of:

[Accurate cell tracking and lineage construction in live-cell imaging experiments with deep learning](https://www.biorxiv.org/content/10.1101/803205v2)

Deployed at:

[DeepCell.org](http://www.deepcell.org/)

### Global Imports

In [1]:
import os
import datetime
import errno
import argparse
import fnmatch

import numpy as np

from skimage.external.tifffile import imsave
from skimage.external.tifffile import TiffFile

import deepcell
from deepcell.utils.misc_utils import sorted_nicely
from deepcell.utils.tracking_utils import load_trks

## Track Multiple Movies with Multiple Parameters

### Load Ground Truth Benchmark Data

In [2]:
# Download four different sets of ground truth data (saves to ~/.keras/datasets)
# We will re-track this GT data to verify our results
filename_3T3 = '3T3_NIH_benchmarks.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.nih_3t3_bench.load_tracked_data(filename_3T3)
print('3T3 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_HEK = 'HEK293_benchmarks.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.hek293_bench.load_tracked_data(filename_HEK)
print('HEK293 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_HeLa = 'HeLa_S3_benchmarks.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.hela_s3_bench.load_tracked_data(filename_HeLa)
print('HeLa -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_RAW = 'RAW2647_benchmarks.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.raw2647_bench.load_tracked_data(filename_RAW)
print('RAW264.7 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

322985984/322979840 [==============================] - 26s 0us/step
3T3 -
X.shape: (19, 30, 154, 182, 1)
y.shape: (19, 30, 154, 182, 1)
134987776/134983680 [==============================] - 7s 0us/step
HEK293 -
X.shape: (20, 30, 135, 160, 1)
y.shape: (20, 30, 135, 160, 1)
637083648/637081600 [==============================] - 23s 0us/step
HeLa -
X.shape: (14, 40, 216, 256, 1)
y.shape: (14, 40, 216, 256, 1)
226959360/226959360 [==============================] - 7s 0us/step
RAW264.7 -
X.shape: (10, 30, 202, 240, 1)
y.shape: (10, 30, 202, 240, 1)


### Setup File Path Constants

In [3]:
# Change DATA_DIR if you are not using `deepcell.datasets`
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))

ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models'))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs'))

# Define destination folders for tracked output
cor_tracked_bens_3T3  = os.path.abspath(os.path.join(ROOT_DIR,'/track_data/3T3/'))
cor_tracked_bens_HEK  = os.path.abspath(os.path.join(ROOT_DIR,'/track_data/HEK293/'))
cor_tracked_bens_HeLa = os.path.abspath(os.path.join(ROOT_DIR,'/track_data/HeLa/'))
cor_tracked_bens_RAW  = os.path.abspath(os.path.join(ROOT_DIR,'/track_data/RAW264.7/'))

TRACK_DIRS = [
    cor_tracked_bens_3T3,
    cor_tracked_bens_HEK,
    cor_tracked_bens_HeLa,
    cor_tracked_bens_RAW
]

# create directories if they do not exist
for d in TRACK_DIRS + [MODEL_DIR, LOG_DIR]:
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

# Define a base file name for the output
BASE_NAME = 'batch_'

In [4]:
# Define data to track (benchmark test files)
# Each trks file contains multiple movies (or batches)
GT_trks_files = [
    os.path.join(DATA_DIR, filename_3T3),
    os.path.join(DATA_DIR, filename_HEK),
    os.path.join(DATA_DIR, filename_HeLa),
    os.path.join(DATA_DIR, filename_RAW)
]

### Work Through Each Dataset with Multiple Parameters

#### Import a Tracking Model to Use

In [5]:
from deepcell import model_zoo

# Tracking model settings (These settings should mirror those from Part 1)
features = {'appearance', 'distance', 'neighborhood', 'regionprop'}
min_track_length = 9
neighborhood_scale_size = 30
batch_size = 128  
crop_dim = 32

in_shape = (crop_dim, crop_dim, 1)

# Re-instantiate the tracking model
tracking_model = model_zoo.siamese_model(
    input_shape=in_shape,
    neighborhood_scale_size=neighborhood_scale_size,
    features=features)

# Load model weights
siamese_weights_file = 'tracking_model_seed1_tl9.h5'
siamese_weights_file = os.path.join(MODEL_DIR, siamese_weights_file)

tracking_model.load_weights(siamese_weights_file)

#### Define Tracking Parameters

In [6]:
# Define Critical Parameters 
# Key variables are lists to facilitate parameter sweeps if desired

# If model prediction for daughter is higher than this parameter,
# then daughter assignment made
division = [0.9]

# Parameter for cost matrix - if other possible assignments are higher than this,
# then a birth occurred
birth = [0.99]

# Parameter for cost matrix - if other possible assignments are higher than this,
# then a death occurred
death = [0.99]

# If two labels are beyond this distance they will not be compared
# (smaller distances -> faster tracking)
max_distance = 50

# Define Parameters that should not be changed
# (they are model dependent or training data dependent)
track_length = 9
neighborhood_scale_size = 30

#### Run the Model Iteratively for the Parameters of Interest

In [7]:
# The tracking model is used in concert with other processes to track cells
# Import the neccesary tracking functionality
import deepcell.tracking

# The tracker requires normalized images as input
def image_norm(original_image):
    # NNs prefer input data that is 0 mean and unit variance
    normed_image = (original_image - np.mean(original_image)) / np.std(original_image)
    return normed_image

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for set_num, dataset in enumerate(GT_trks_files):
    trks = load_trks(dataset)
    # Go through each batch (movie) in each dataset
    for batch_num, (lineage, raw, tracked) in enumerate(zip(trks['lineages'], trks['X'], trks['y'])):        
        # Normalize raw images
        for frame in range(raw.shape[0]):
            raw[frame, :, :, 0] = image_norm(raw[frame, :, :, 0]) 

        # Track each movie with each division parameter
        print('Tracking: Dataset {}, Batch {}'.format(set_num, batch_num))
        for div_param in division:
            # For each birth parameter
            for birth_param in birth:
                # For each death parameter
                for death_param in death:
                    # Track with the selected parameters
                    trial = deepcell.tracking.cell_tracker(raw, tracked,
                                         tracking_model, max_distance=max_distance, track_length=track_length, 
                                         division=div_param, birth=birth_param, death=death_param,
                                         neighborhood_scale_size=neighborhood_scale_size,
                                         features=features)

                    trial.track_cells()
                    
                    # Run FP post processing and save the result in the correct location
                    file_name = BASE_NAME + str(batch_num).zfill(3)+'_div'+str(div_param).zfill(3)+'_b'+str(birth_param).zfill(3)+'_d'+str(death_param).zfill(3)+'.trk'
                    file_path = os.path.join(TRACK_DIRS[set_num], file_name)
                    fp_fixed_trk = trial.postprocess(file_path)


Tracking: Dataset 0, Batch 0
Tracking frame 1


/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


Tracked frame 1 in 9.948533233022317 seconds.
Tracking frame 2
Tracked frame 2 in 0.25320231600198895 seconds.
Tracking frame 3
Tracked frame 3 in 0.16349016805179417 seconds.
Tracking frame 4
Tracked frame 4 in 0.16464403399731964 seconds.
Tracking frame 5
Tracked frame 5 in 0.17371689900755882 seconds.
Tracking frame 6
New track
Tracked frame 6 in 0.26787602400872856 seconds.
Tracking frame 7
Tracked frame 7 in 0.2626428749645129 seconds.
Tracking frame 8
Tracked frame 8 in 0.17699549393728375 seconds.
Tracking frame 9
Tracked frame 9 in 0.18178974010515958 seconds.
Tracking frame 10
Tracked frame 10 in 0.18032974598463625 seconds.
Tracking frame 11
New track
Division detected
New track
Division detected
Tracked frame 11 in 0.2564015599200502 seconds.
Tracking frame 12
New track
Division detected
New track
Division detected
Tracked frame 12 in 0.3162414829712361 seconds.
Tracking frame 13
Tracked frame 13 in 0.35619708709418774 seconds.
Tracking frame 14
Tracked frame 14 in 0.2205645

Tracked frame 5 in 0.11527215398382396 seconds.
Tracking frame 6
Tracked frame 6 in 0.11710570903960615 seconds.
Tracking frame 7
New track
Tracked frame 7 in 0.12544648197945207 seconds.
Tracking frame 8
Tracked frame 8 in 0.13133054599165916 seconds.
Tracking frame 9
Tracked frame 9 in 0.1307360779028386 seconds.
Tracking frame 10
Tracked frame 10 in 0.1316881129750982 seconds.
Tracking frame 11
Tracked frame 11 in 0.13212226203177124 seconds.
Tracking frame 12
Tracked frame 12 in 0.12143785902298987 seconds.
Tracking frame 13
Tracked frame 13 in 0.1305933069670573 seconds.
Tracking frame 14
Tracked frame 14 in 0.1209863219410181 seconds.
Tracking frame 15
Tracked frame 15 in 0.12828317005187273 seconds.
Tracking frame 16
Tracked frame 16 in 0.13076426496263593 seconds.
Tracking frame 17
Tracked frame 17 in 0.12997670006006956 seconds.
Tracking frame 18
Tracked frame 18 in 0.1279636810068041 seconds.
Tracking frame 19
New track
Division detected
New track
Division detected
Tracked fr

Tracked frame 9 in 0.16773275297600776 seconds.
Tracking frame 10
Tracked frame 10 in 0.16201139893382788 seconds.
Tracking frame 11
Tracked frame 11 in 0.1642872199881822 seconds.
Tracking frame 12
Tracked frame 12 in 0.16936187201645225 seconds.
Tracking frame 13
Tracked frame 13 in 0.18637169792782515 seconds.
Tracking frame 14
Tracked frame 14 in 0.1743904349859804 seconds.
Tracking frame 15
Tracked frame 15 in 0.16374942695256323 seconds.
Tracking frame 16
Tracked frame 16 in 0.16463161795400083 seconds.
Tracking frame 17
Tracked frame 17 in 0.1648128730012104 seconds.
Tracking frame 18
Tracked frame 18 in 0.16377342597115785 seconds.
Tracking frame 19
Tracked frame 19 in 0.1739601050503552 seconds.
Tracking frame 20
Tracked frame 20 in 0.17495091201271862 seconds.
Tracking frame 21
Tracked frame 21 in 0.16352466691751033 seconds.
Tracking frame 22
Tracked frame 22 in 0.14039962505921721 seconds.
Tracking frame 23
Tracked frame 23 in 0.1329957420239225 seconds.
Tracking frame 24
T

...

Tracked frame 27 in 0.09611483302433044 seconds.
Tracking frame 28
Tracked frame 28 in 0.0917295649414882 seconds.
Tracking frame 29
Tracked frame 29 in 0.09308954607695341 seconds.
Tracking frame 30
Tracked frame 30 in 0.09281182801350951 seconds.
Tracking frame 31
Tracked frame 31 in 0.09280772402416915 seconds.
Tracking frame 32
New track
Tracked frame 32 in 0.10644165007397532 seconds.
Tracking frame 33
Tracked frame 33 in 0.10119812400080264 seconds.
Tracking frame 34
Tracked frame 34 in 0.09709028794895858 seconds.
Tracking frame 35
Tracked frame 35 in 0.09382068400736898 seconds.
Tracking frame 36
Tracked frame 36 in 0.0961275709560141 seconds.
Tracking frame 37
Tracked frame 37 in 0.09341785404831171 seconds.
Tracking frame 38
Tracked frame 38 in 0.11042116302996874 seconds.
Tracking frame 39
Tracked frame 39 in 0.09848405292723328 seconds.
Tracking: Dataset 2, Batch 15
Tracking frame 1
Tracked frame 1 in 0.2562141730450094 seconds.
Tracking frame 2
Tracked frame 2 in 0.2522198

Tracked frame 31 in 0.07210118393413723 seconds.
Tracking frame 32
Tracked frame 32 in 0.07102763303555548 seconds.
Tracking frame 33
Tracked frame 33 in 0.07239936909172684 seconds.
Tracking frame 34
Tracked frame 34 in 0.07455802103504539 seconds.
Tracking frame 35
Tracked frame 35 in 0.07619361602701247 seconds.
Tracking frame 36
Tracked frame 36 in 0.07354231202043593 seconds.
Tracking frame 37
Tracked frame 37 in 0.07443855807650834 seconds.
Tracking frame 38
Tracked frame 38 in 0.07312287797685713 seconds.
Tracking frame 39
Tracked frame 39 in 0.06631593394558877 seconds.
Tracking: Dataset 3, Batch 0
Tracking frame 1
New track
Tracked frame 1 in 0.18134737003128976 seconds.
Tracking frame 2
New track
Tracked frame 2 in 0.18181802099570632 seconds.
Tracking frame 3
Tracked frame 3 in 0.18963507609441876 seconds.
Tracking frame 4
Tracked frame 4 in 0.1895224479958415 seconds.
Tracking frame 5
Tracked frame 5 in 0.17198492598254234 seconds.
Tracking frame 6
Tracked frame 6 in 0.1699

Tracked frame 19 in 0.1305948810186237 seconds.
Tracking frame 20
Tracked frame 20 in 0.12761912192218006 seconds.
Tracking frame 21
New track
Tracked frame 21 in 0.13991974003147334 seconds.
Tracking frame 22
Tracked frame 22 in 0.14769825094845146 seconds.
Tracking frame 23
Tracked frame 23 in 0.1536226369207725 seconds.
Tracking frame 24
Tracked frame 24 in 0.15883708000183105 seconds.
Tracking frame 25
Tracked frame 25 in 0.14623844099696726 seconds.
Tracking frame 26
Tracked frame 26 in 0.1494545149616897 seconds.
Tracking frame 27
New track
Tracked frame 27 in 0.15916734805796295 seconds.
Tracking frame 28
Tracked frame 28 in 0.1635525260353461 seconds.
Tracking frame 29
Tracked frame 29 in 0.13349882897455245 seconds.
Tracking: Dataset 3, Batch 4
Tracking frame 1
Tracked frame 1 in 0.368073643068783 seconds.
Tracking frame 2
New track
New track
Tracked frame 2 in 0.3868682460160926 seconds.
Tracking frame 3
Tracked frame 3 in 0.3750410620123148 seconds.
Tracking frame 4
Tracked 

Tracked frame 20 in 0.11203040299005806 seconds.
Tracking frame 21
Tracked frame 21 in 0.1284093800932169 seconds.
Tracking frame 22
Tracked frame 22 in 0.13421595096588135 seconds.
Tracking frame 23
Tracked frame 23 in 0.1303749579237774 seconds.
Tracking frame 24
Tracked frame 24 in 0.13150477700401098 seconds.
Tracking frame 25
Tracked frame 25 in 0.13346472394187003 seconds.
Tracking frame 26
Tracked frame 26 in 0.133469954947941 seconds.
Tracking frame 27
Tracked frame 27 in 0.13143386703450233 seconds.
Tracking frame 28
Tracked frame 28 in 0.12116805103141814 seconds.
Tracking frame 29
Tracked frame 29 in 0.09714928304310888 seconds.
Tracking: Dataset 3, Batch 8
Tracking frame 1
Tracked frame 1 in 0.22003534098621458 seconds.
Tracking frame 2
Tracked frame 2 in 0.2081744250608608 seconds.
Tracking frame 3
New track
Division detected
New track
Division detected
New track
Division detected
Tracked frame 3 in 0.2253830940462649 seconds.
Tracking frame 4
Tracked frame 4 in 0.22171918

Tracked frame 21 in 0.5826423580292612 seconds.
Tracking frame 22
Tracked frame 22 in 0.5981117549818009 seconds.
Tracking frame 23
New track
Division detected
New track
Division detected
Tracked frame 23 in 0.6269145209807903 seconds.
Tracking frame 24
Tracked frame 24 in 0.6840144729940221 seconds.
Tracking frame 25
Tracked frame 25 in 0.6256222008960322 seconds.
Tracking frame 26
Tracked frame 26 in 0.6941595940152183 seconds.
Tracking frame 27
Tracked frame 27 in 0.6564545260043815 seconds.
Tracking frame 28
Tracked frame 28 in 0.6248858988983557 seconds.
Tracking frame 29
Tracked frame 29 in 0.5286794239655137 seconds.
Tracking: Dataset 3, Batch 12
Tracking frame 1
New track
Tracked frame 1 in 0.14958351105451584 seconds.
Tracking frame 2
Tracked frame 2 in 0.1500024130800739 seconds.
Tracking frame 3
Tracked frame 3 in 0.1502278969855979 seconds.
Tracking frame 4
New track
Division detected
New track
Division detected
New track
Division detected
New track
Division detected
Tracke

## Translate the Results Into ISBI Format for Comparison

#### Save Cell Lineages in an ISBI-Formatted Output txt

The ISBI Cell Tracking Challenge requires a text file (man_track.txt) that represents a batch's cell lineage as an acyclic graph. The format of this file is as follows: Every line corresponds to a single track that is encoded by four numbers separated by a space -  
L B E P  
where L is a unique label of the track (label of markers, 16-bit positive value),  
B is a zero-based index of the frame in which the track begins,  
E is a zero-based index of the frame in which the track ends,  
P is the label of the parent track (0 is used when no parent is defined)

N.B. DeepCell's unique approach allows for cells to be tracked even if it momentarily leaves the frame. This is not possible in convential tracking algorithms, so ISBI considers a cell's track to have ended once it leaves the frame. We adjust the output here to keep with ISBI's formatting (ie. each track only contains contiguous frames).

#### ISBI Translator Functions

In [8]:
from deepcell_tracking.isbi_utils import contig_tracks
from deepcell_tracking.isbi_utils import create_new_ISBI_track

#### Define Save Location for Benchmark Files (ie: 001, 001_GT, 001_RES)

In [9]:
# Define where benchmark data will be saved
BENCH_DIR = os.path.abspath(os.path.join(ROOT_DIR, '/tracking_benchmarks'))

# Define where tracks will be saved
bench_3T3  = os.path.join(BENCH_DIR, '3T3/')
bench_HEK  = os.path.join(BENCH_DIR, 'HEK293/')
bench_HeLa = os.path.join(BENCH_DIR, 'HeLa/')
bench_RAW  = os.path.join(BENCH_DIR, 'RAW264.7/')

BENCH_DIRS = [
    bench_3T3,
    bench_HEK,
    bench_HeLa,
    bench_RAW
]

# create directories if they do not exist
for d in BENCH_DIRS:
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

#### Generate RAW and GT Benchmark Files

In [11]:
# Create benchmark files (ie: 001, 001_GT, etc) from the Ground Truth .trk files

from skimage.external.tifffile import imsave

for set_num, dataset in enumerate(GT_trks_files):
    # Load trks file
    trks = load_trks(dataset)
    lineages, raw, tracked = trks["lineages"], trks["X"], trks["y"]
       
    # Define Save Location
    PARAMS = 'div'+str(div_param).zfill(3)+'_b'+str(birth_param).zfill(3)+'_d'+str(death_param).zfill(3)

    # Build subdirectories to hold benchmark info           
    BENCHMARK_DIR = os.path.abspath(os.path.join(BENCH_DIRS[set_num], PARAMS))
    
    # Record lineage data in txt as it is generated
    for batch, batch_info in enumerate(lineages):
        print('batch number: ', batch)

        # Build subdirectories to hold benchmark info
        B_SUB_DIR_RAW = os.path.join(BENCHMARK_DIR, '{:03}'.format(batch+1))
        B_SUB_DIR_GT = os.path.join(BENCHMARK_DIR, '{:03}_GT'.format(batch+1))
        B_SUB_DIR_SEG = os.path.join(B_SUB_DIR_GT, 'SEG')
        B_SUB_DIR_TRA = os.path.join(B_SUB_DIR_GT, 'TRA')

        # Create directories if they do not exist
        for d in (B_SUB_DIR_RAW, B_SUB_DIR_GT, B_SUB_DIR_SEG, B_SUB_DIR_TRA):
            try:
                os.makedirs(d)
            except OSError as exc:  # Guard against race condition
                if exc.errno != errno.EEXIST:
                    raise

        # Prepare output txt
        text_file = open(os.path.join(B_SUB_DIR_TRA, "man_track.txt"), "w")

        batch_tracked = tracked[batch]
        labels = list(batch_info.keys())
        max_label = max(labels)

        labels_to_remove = []
        for label in labels:
            batch_info, batch_tracked = contig_tracks(label, batch_info, batch_tracked)
            first_frame = np.amin(batch_info[label]['frames'])          
            last_frame = np.amax(batch_info[label]['frames'])
            if batch_info[label]['parent']:
                parent = batch_info[label]['parent']
            else:
                parent = 0

            print(label, first_frame, last_frame, parent)
            text_file.write('{} {} {} {}\n'.format(label, first_frame, last_frame, parent))

            # Check if the track need to be split
            if max(batch_info.keys()) > max_label:
                # If so, a new track was added
                new_max_label = max(batch_info.keys())
                labels.append(new_max_label)
                max_label = new_max_label

        text_file.close()
        lineages[batch] = batch_info
        tracked[batch] = batch_tracked
        
        # Generate new images to match the new ISBI-formatted lineage data

        channel = 0 # These images should only have one channel
        for i in range(raw.shape[1]):
            name_raw = os.path.join(B_SUB_DIR_RAW,'t{:03}_.tif'.format(i))
            name_tracked_SEG = os.path.join(B_SUB_DIR_SEG,'man_seg{:03}.tif'.format(i))
            name_tracked_TRA = os.path.join(B_SUB_DIR_TRA,'man_track{:03}.tif'.format(i))
            imsave(name_raw, raw[batch, i, :, :, channel])
            imsave(name_tracked_SEG, tracked[batch, i, :, :, channel].astype('uint16'))
            imsave(name_tracked_TRA, tracked[batch, i, :, :, channel].astype('uint16'))    
        

batch number:  0
1 0 29 0
2 0 29 0
3 0 29 0
4 0 10 0
5 11 29 4
6 11 29 4
7 0 29 0
8 0 11 0
9 12 29 8
10 12 29 8
11 0 29 0
12 0 29 0
13 0 29 0
14 6 10 0
15 19 29 0
16 12 22 0
17 25 29 0
batch number:  1
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 28 0
6 0 12 0
7 15 29 0
8 13 13 6
9 13 29 6
batch number:  2
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 29 0
9 0 29 0
10 20 29 0
11 16 29 0
batch number:  3
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 4 29 17
6 0 29 0
7 0 29 0
8 0 29 0
9 0 29 0
10 0 26 0
11 0 29 0
12 4 29 17
13 3 4 0
14 8 29 0
15 9 18 0
16 0 29 0
17 0 3 0
18 6 26 0
19 29 29 0
batch number:  4
1 0 18 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 12 0
7 7 29 0
8 19 29 1
9 19 29 1
10 13 29 6
batch number:  5
1 0 16 0
2 17 29 1
3 17 29 1
4 0 29 0
5 0 29 0
6 0 27 0
7 17 29 0
8 0 29 0
9 0 29 0
batch number:  6
1 0 29 0
2 0 1 0
3 2 29 2
4 0 29 0
5 0 22 0
6 0 29 0
7 0 29 0
8 0 29 0
9 1 28 0
10 2 29 2
11 25 29 0
batch number:  7
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0

25 14 15 0
26 24 29 5
28 26 29 13
29 26 29 13
30 24 29 5
31 1 10 0
32 10 14 0
33 15 28 0
34 13 29 0
35 18 29 0
36 18 20 0
37 16 16 0
38 28 29 0
39 18 18 0
batch number:  16
1 0 18 0
2 0 18 0
3 0 29 0
4 0 16 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 29 0
9 0 29 0
10 0 29 0
11 0 29 0
12 0 29 0
13 0 29 0
14 0 29 0
15 0 29 0
16 0 29 0
17 0 12 0
18 0 29 0
19 5 5 0
20 13 29 17
21 17 29 4
23 21 21 0
24 17 29 4
25 13 18 17
26 27 28 0
27 29 29 0
batch number:  17
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 0 0
9 0 29 0
10 0 20 0
11 0 29 0
12 0 29 0
13 0 29 0
14 0 29 0
15 0 13 0
16 0 29 0
17 0 13 0
18 0 29 0
19 11 13 0
20 14 29 15
22 21 29 0
23 21 29 10
25 21 29 10
26 14 29 17
27 14 29 17
28 4 5 0
29 14 29 15
30 16 16 0
31 7 9 0
32 14 29 0
batch number:  18
1 0 29 0
2 0 29 0
3 0 10 0
4 0 29 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 29 0
9 0 17 0
10 0 29 0
11 0 13 0
12 0 29 0
13 0 29 0
14 0 29 0
15 0 29 0
16 0 29 0
17 0 22 0
18 0 29 0
19 0 15 0
20 0 29 0
21 2 29 0
23 12 29 0
24 20 29 0
25 19

#### Translate Result trk files and Generate Benchmark Files

In [12]:
## Create new benchmark files (ie: 001_RES) from mutliple .trk files

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for set_num, dataset in enumerate(TRACK_DIRS):
    # Go through each batch (movie) in each dataset
    movie_list = sorted_nicely(os.listdir(dataset))
    
    for batch_num, batch in enumerate(movie_list):

        # Load the trk file
        filename = os.path.join(dataset, batch)
        trks = load_trks(filename)
        lineages, raw, tracked = trks["lineages"], trks["X"], trks["y"]

        # Define Save Location
        PARAMS = 'div'+str(div_param).zfill(3)+'_b'+str(birth_param).zfill(3)+'_d'+str(death_param).zfill(3)
        
        # Build subdirectories to hold benchmark info           
        BENCHMARK_DIR = os.path.abspath(os.path.join(BENCH_DIRS[set_num], PARAMS))
        B_SUB_DIR = os.path.join(BENCHMARK_DIR, '{:03}_RES'.format(batch_num+1))

        print('Files will be saved at: ', B_SUB_DIR)

        # create directories if they do not exist
        try:
            os.makedirs(B_SUB_DIR)
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

        # Record lineage data in txt as it is generated
        batch_info = lineages[0]

        # Prepare output txt
        text_file = open(os.path.join(B_SUB_DIR, "res_track.txt"), "w")

        batch_tracked = tracked
        labels = list(batch_info.keys())
        max_label = max(labels)

        labels_to_remove = []
        for label in labels:

            batch_info, batch_tracked = contig_tracks(label, batch_info, batch_tracked)

            first_frame = np.amin(batch_info[label]['frames'])          
            last_frame = np.amax(batch_info[label]['frames'])
            if batch_info[label]['parent']:
                parent = batch_info[label]['parent']
            else:
                parent = 0

            print(label, first_frame, last_frame, parent)
            text_file.write('{} {} {} {}\n'.format(label, first_frame, last_frame, parent))

            # Check if the track need to be split
            if max(batch_info.keys()) > max_label:
                # If so, a new track was added
                new_max_label = max(batch_info.keys())
                labels.append(new_max_label)
                max_label = new_max_label

        text_file.close()
        lineages[0] = batch_info
        tracked = batch_tracked


        # Save Image Files
        channel = 0 # These images should only have one channel
        for i in range(raw.shape[0]):
            name_tracked = os.path.join(B_SUB_DIR,'mask{:03}.tif'.format(i))
            imsave(name_tracked, tracked[i, :, :, channel].astype('uint16'))

Files will be saved at:  /tracking_benchmarks/3T3/div0.9_b0.99_d0.99/001_RES
1 0 29 0
2 0 29 0
3 0 29 0
4 0 10 0
5 0 29 0
6 0 11 0
7 0 29 0
8 0 29 0
9 0 29 0
10 6 10 0
11 11 29 4
12 11 29 4
13 12 29 6
14 12 29 6
15 19 29 0
16 12 22 0
17 25 29 0
Files will be saved at:  /tracking_benchmarks/3T3/div0.9_b0.99_d0.99/002_RES
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 28 0
6 0 12 0
7 13 13 6
8 13 29 6
9 15 29 0
Files will be saved at:  /tracking_benchmarks/3T3/div0.9_b0.99_d0.99/003_RES
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 29 0
9 0 29 0
10 16 29 0
11 20 29 0
Files will be saved at:  /tracking_benchmarks/3T3/div0.9_b0.99_d0.99/004_RES
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 29 0
9 0 26 0
10 0 29 0
11 0 29 0
12 0 3 0
13 3 4 0
14 4 29 12
15 4 29 12
16 8 29 0
17 9 18 0
18 6 26 0
19 29 29 0
Files will be saved at:  /tracking_benchmarks/3T3/div0.9_b0.99_d0.99/005_RES
1 0 18 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 29 0
7 7 29 0
8 19 29 1
9 19 

Files will be saved at:  /tracking_benchmarks/HEK293/div0.9_b0.99_d0.99/013_RES
1 0 29 0
2 0 29 0
3 0 16 0
4 0 29 0
5 0 29 0
6 0 15 0
7 0 29 0
8 0 29 0
9 0 29 0
10 0 29 0
11 0 29 0
12 0 29 0
13 0 29 0
14 0 29 0
15 17 29 3
16 17 29 3
18 17 29 3
Files will be saved at:  /tracking_benchmarks/HEK293/div0.9_b0.99_d0.99/014_RES
1 0 29 0
2 0 13 0
3 0 29 0
4 0 29 0
5 0 15 0
6 0 29 0
7 0 13 0
8 0 29 0
9 0 29 0
10 0 29 0
11 0 29 0
12 0 6 0
13 0 29 0
14 0 23 0
15 0 11 0
16 0 29 0
17 0 0 0
18 0 29 0
19 4 29 0
20 5 29 0
21 7 29 12
22 7 29 12
23 14 29 7
24 14 29 7
25 16 18 5
26 16 29 5
27 25 29 0
28 16 23 0
29 19 20 0
30 7 8 0
31 28 29 0
Files will be saved at:  /tracking_benchmarks/HEK293/div0.9_b0.99_d0.99/015_RES
1 0 13 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 29 0
9 0 29 0
10 0 21 0
11 0 29 0
12 0 0 0
13 0 29 0
14 0 29 0
15 0 29 0
16 0 4 0
17 0 29 0
18 0 29 0
19 0 4 0
20 2 10 0
21 2 29 0
22 4 29 0
23 22 29 10
24 22 29 10
25 23 29 0
26 2 13 0
27 7 25 0
28 7 8 0
29 26 26 0
30 16

Files will be saved at:  /tracking_benchmarks/RAW264.7/div0.9_b0.99_d0.99/003_RES
1 0 23 0
2 0 22 0
3 0 29 0
4 0 25 0
5 0 29 0
6 0 29 0
7 0 29 0
8 0 29 0
9 0 29 0
10 0 29 0
11 0 29 0
12 0 29 0
13 0 29 0
14 0 17 0
15 0 9 0
16 0 29 0
17 1 29 0
18 23 29 2
19 23 29 2
20 18 18 0
Files will be saved at:  /tracking_benchmarks/RAW264.7/div0.9_b0.99_d0.99/004_RES
1 0 29 0
2 0 29 0
3 0 29 0
4 0 29 0
5 0 29 0
6 0 29 0
7 21 29 0
8 27 29 0
Files will be saved at:  /tracking_benchmarks/RAW264.7/div0.9_b0.99_d0.99/005_RES
1 0 0 0
2 0 29 0
3 0 21 0
4 0 29 0
5 0 19 0
6 0 29 0
7 0 16 0
8 0 29 0
9 0 29 0
10 0 29 0
11 0 29 0
12 0 29 0
13 0 11 0
14 0 29 0
15 0 17 0
16 0 29 0
17 0 1 0
18 2 2 0
19 2 2 0
20 12 29 13
21 12 29 13
22 17 20 7
23 17 24 7
24 18 18 15
25 18 29 15
26 20 29 5
27 20 29 5
29 22 29 3
30 22 29 3
31 27 29 0
32 28 29 0
33 2 7 0
34 4 29 0
35 13 13 0
36 26 29 0
37 21 23 0
38 9 9 0
39 25 29 0
Files will be saved at:  /tracking_benchmarks/RAW264.7/div0.9_b0.99_d0.99/006_RES
1 0 3 0
2 0 29 0
3 0

## Run Graph Benchmarks

#### Graph Comparison Code

In [13]:
from deepcell_tracking.isbi_utils import txt_to_graph
from deepcell_tracking.isbi_utils import classify_divisions

from deepcell.metrics import match_nodes

#### Iteratively Compare Each Movie (Tracked with Each Parameter) To the Ground Truth Data

In [15]:
# Prep a list to hold each dataset's results
dataset_cm = []

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for ben_trk_folder in BENCH_DIRS:

    # Prep a list to hold results for each set of parameters
    params_cm = []
   
    # Go through each parameter combination
    for div_param in division:
        # For each birth parameter
        for birth_param in birth:
            # For each death parameter
            for death_param in death:  

                PARAMS = 'div'+str(div_param)+'_b'+str(birth_param)+'_d'+str(death_param)
                BENCHMARK_DIR = os.path.join(ben_trk_folder, PARAMS)
                
                # Compile a list of each movie
                sub_dirs = sorted_nicely(os.listdir(BENCHMARK_DIR))
                movie_list = fnmatch.filter(sub_dirs, '???')
                # Prep a list to hold each movie's results
                cm_list = []
                # Loop through each set of movies 
                for name in movie_list:
                    # Extract track.txt for each movie
                    pattern_gt = os.path.join(BENCHMARK_DIR, name + '_GT/TRA/')
                    pattern_res = os.path.join(BENCHMARK_DIR, name + '_RES/')

                    gtcells, rescells = match_nodes(pattern_gt+'*.tif',pattern_res+'*.tif')

                    if len(np.unique(rescells)) < len(np.unique(gtcells)):
                        node_key = {r:g for g,r in zip(gtcells,rescells)}
                        # node_key
                        # Maps gt nodes onto resnodes so must be applied to gt
                        G_res = txt_to_graph(pattern_res+'res_track.txt',node_key=node_key)
                        G_gt = txt_to_graph(pattern_gt+'man_track.txt')
                        cm_list.append(classify_divisions(G_gt,G_res))
                    else:
                        node_key = {g:r for g,r in zip(gtcells,rescells)}
                        G_res = txt_to_graph(pattern_res+'res_track.txt')
                        G_gt = txt_to_graph(pattern_gt+'man_track.txt',node_key=node_key)
                        cm_list.append(classify_divisions(G_gt,G_res))            

                params_cm.append(cm_list)
                
    dataset_cm.append(params_cm)

attempted removal of node 6_12 failed
attempted removal of node 12_5 failed
attempted removal of node 4_20 failed
attempted removal of node 17_13 failed
attempted removal of node 13_25 failed
attempted removal of node 5_36 failed
attempted removal of node 8_7 failed
attempted removal of node 1_17 failed


In [16]:
cm_totals = []
for cell_type in dataset_cm:
    cm_params = []
    for param_comb in cell_type:

        divI, divJ, divC, divGH = 0, 0, 0, 0
        dataset_stats = {
            'Correct division': divI,
            'Incorrect division': divJ,
            'False positive division': divC,
            'False negative division': divGH
        }

        for cm in param_comb:
            dataset_stats['Correct division'] = dataset_stats['Correct division']+cm['Correct division']
            dataset_stats['Incorrect division'] = dataset_stats['Incorrect division']+cm['Incorrect division']
            dataset_stats['False positive division'] = dataset_stats['False positive division']+cm['False positive division']
            dataset_stats['False negative division'] = dataset_stats['False negative division']+cm['False negative division']

        cm_params.append(dataset_stats)
    
    cm_totals.append(cm_params)

In [17]:
counter = 0
# Go through each parameter combination
for div_param in division:
    # For each birth parameter
    for birth_param in birth:
        # For each death parameter
        for death_param in death:  

            if counter == 0:
                PARAMS = 'div'+str(div_param)+'_b'+str(birth_param)+'_d'+str(death_param)
                print(PARAMS)
                print('3T3')
                print(cm_totals[0][counter])
                print('HEK293')
                print(cm_totals[1][counter])
                print('HeLa')
                print(cm_totals[2][counter])
                print('RAW')
                print(cm_totals[3][counter])

            counter = counter + 1

div0.9_b0.99_d0.99
3T3
{'False negative division': 4, 'False positive division': 0, 'Incorrect division': 0, 'Correct division': 18}
HEK293
{'False negative division': 1, 'False positive division': 7, 'Incorrect division': 0, 'Correct division': 36}
HeLa
{'False negative division': 1, 'False positive division': 0, 'Incorrect division': 0, 'Correct division': 8}
RAW
{'False negative division': 2, 'False positive division': 5, 'Incorrect division': 0, 'Correct division': 21}


## Run ISBI Benchmarking Scripts 

#### ISBI Cell Tracking Challenge Software Required 

ISBI's Cell Tracking Challenge has a specific [evaluation methodolgy](http://celltrackingchallenge.net/evaluation-methodology/). The evaluation package can be downloaded [here](http://public.celltrackingchallenge.net/software/EvaluationSoftware.zip). Extract the TRAMeasure executable from the zip file and place it in the same directory as this notebook.

In [18]:
import subprocess
import statistics

# Confirm the ISBI CTC executable file is available
assert os.path.isfile('./TRAMeasure'),'CTC EXE unavailable. See subheading for download instructions.'

# Define path to folder containing the directories of interest (ie: 001, 001_GT, 001_RES)
bens_folder_names = ['3T3', 'HEK293', 'HeLa', 'RAW264.7']

for index, path in enumerate(BENCH_DIRS):
    
    # Check into parameter folder if neccesary
    PARAMS = 'div'+str(div_param).zfill(3)+'_b'+str(birth_param).zfill(3)+'_d'+str(death_param).zfill(3)
    path = os.path.abspath(os.path.join(path, PARAMS))

    # Calculate the number of batches
    dirs = os.listdir(path)
    num_batches = int(len(dirs)/3)

    TRA_Vals = []
    for batch in range(num_batches):
        batch = '{:03}'.format(batch+1)

        # Run ISBI Tracking Benchmark
        p = subprocess.run(['./TRAMeasure', path, batch], stdout=subprocess.PIPE)
        # Save the output
        outstring = p.stdout

        try:
            TRA_Val = float(outstring.decode('utf-8').split()[-1])
            TRA_Vals.append(TRA_Val)
        except:
            print('Benchmarking failure - Batch ', batch)
            print(outstring.decode('utf-8'))
    
    print('Dataset: ', bens_folder_names[index])
    print('Mean: ', statistics.mean(TRA_Vals))
    print('Std. Dev.: ', statistics.pstdev(TRA_Vals))

Dataset:  3T3
Mean:  0.9998386666666667
Std. Dev.:  0.00034323675826202356
Dataset:  HEK293
Mean:  0.9995211538461538
Std. Dev.:  0.0005735275925430942
Dataset:  HeLa
Mean:  0.9999588333333334
Std. Dev.:  0.00011779466597997058
Dataset:  RAW264.7
Mean:  0.9996243846153846
Std. Dev.:  0.00037403178360660233
